In [12]:
import requests
from bs4 import BeautifulSoup
import pandas

base_url = "http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/#t=0&s="

alist = []

for page in range(0, 30, 10):
    r = requests.get(
        base_url + str(page),
        headers={
            'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'
        }
    )
    soup = BeautifulSoup(r.content, 'html.parser')
    #print(soup.prettify())
    divs = soup.find_all('div', {'class': 'propertyRow'})

    for item in divs:
        prop = {}
        prop['Price'] = item.find('h4', {'class': 'propPrice'}).text.strip()
        prop['Address'] = item.find_all('span', {'class': 'propAddressCollapse'})[0].text
        prop['Locality'] = item.find_all('span', {'class': 'propAddressCollapse'})[1].text
        prop['Beds'] = item.find('span', {'class': 'infoBed'}).find('b').text if item.find('span', {'class': 'infoBed'}) else None
        prop['Area'] = item.find('span', {'class': 'infoSqFt'}).find('b').text if item.find('span', {'class': 'infoSqFt'}) else None
        prop['Full baths'] = item.find('span', {'class': 'infoValueFullBath'}).find('b').text if item.find('span', {'class': 'infoValueFullBath'}) else None
        prop['Half baths'] = item.find('span', {'class': 'infoValueHalfBath'}).find('b').text if item.find('span', {'class': 'infoValueHalfBath'}) else None

        for column_group in item.find_all('div', {'class': 'columnGroup'}):
            for feature_group, feature_name in zip(
                column_group.find_all('span', {'class': 'featureGroup'}),
                column_group.find_all('span', {'class': 'featureName'})
            ):
                if "Lot Size" in feature_group.text:
                    prop['Lot size'] = feature_name.text
        alist.append(prop)


In [13]:
df = pandas.DataFrame(alist)
df

In [15]:
df.to_csv('real_estate.csv')